In [8]:
!pip install PyDrive

In [0]:
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
download = drive.CreateFile({'id':'1t19mxKOrFgePTGOvoCyaB-bj4EguQ4Cq'})

In [0]:
download.GetContentFile('train-scene-classification.zip')
!unzip train-scene-classification.zip

In [0]:
import keras
from keras.models import Sequential
from keras.applications import ResNet50
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from tqdm import tqdm   #for progress bar

In [0]:
train = pd.read_csv('train.csv')


In [26]:
train_image = []
for i in tqdm(range(train.shape[0])):
    img = image.load_img('train/'+train['image_name'][i], target_size=(50,50,3), grayscale=False)
    img = image.img_to_array(img)
    img = img/255
    train_image.append(img)
X = np.array(train_image)

100%|██████████| 17034/17034 [00:17<00:00, 988.88it/s]


In [0]:
y=train['label'].values
y = to_categorical(y)


In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

In [29]:
model=Sequential()
model.add(Conv2D(filters=16,kernel_size=2,padding="same",activation="relu",input_shape=(50,50,3)))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=32,kernel_size=2,padding="same",activation="relu"))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=64,kernel_size=2,padding="same",activation="relu"))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=64,kernel_size=2,padding="same",activation="relu"))
model.add(MaxPooling2D(pool_size=2))
model.add(Flatten())
model.add(Dense(500,activation="relu"))
model.add(Dense(6,activation="softmax"))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 50, 50, 16)        208       
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 25, 25, 16)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 25, 25, 32)        2080      
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 12, 12, 64)        8256      
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 6, 6, 64)          0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 6, 6, 64)          16448     
__________

In [0]:
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])

In [31]:
model.fit(X_train, y_train, epochs=40, validation_data=(X_test, y_test))

Instructions for updating:
Use tf.cast instead.
Train on 13627 samples, validate on 3407 samples
Epoch 1/40
13627/13627 [==============================] - 10s 711us/step - loss: 1.0601 - acc: 0.5891 - val_loss: 0.8213 - val_acc: 0.6815
Epoch 2/40
13627/13627 [==============================] - 5s 358us/step - loss: 0.7445 - acc: 0.7228 - val_loss: 0.6591 - val_acc: 0.7643
Epoch 3/40
13627/13627 [==============================] - 5s 355us/step - loss: 0.6576 - acc: 0.7591 - val_loss: 0.6061 - val_acc: 0.7763
Epoch 4/40
13627/13627 [==============================] - 5s 353us/step - loss: 0.5742 - acc: 0.7898 - val_loss: 0.5640 - val_acc: 0.7913
Epoch 5/40
13627/13627 [==============================] - 5s 349us/step - loss: 0.5377 - acc: 0.8065 - val_loss: 0.5770 - val_acc: 0.7878
Epoch 6/40
13627/13627 [==============================] - 5s 348us/step - loss: 0.4910 - acc: 0.8252 - val_loss: 0.5561 - val_acc: 0.7901
Epoch 7/40
13627/13627 [==============================] - 5s 346us/step - 

In [0]:
download = drive.CreateFile({'id': '1Wpr31a6vIK4NdPMWgcovnNIVkKo9esOU'})
download.GetContentFile('test_WyRytb0.csv')

In [33]:
test = pd.read_csv('test_WyRytb0.csv')
print(test.shape)

(7301, 1)


In [35]:
test_image = []
for i in tqdm(range(test.shape[0])):
    img = image.load_img('train/'+train['image_name'][i], target_size=(50,50,3), grayscale=False)
    img = image.img_to_array(img)
    img = img/255
    test_image.append(img)
X = np.array(test_image)

100%|██████████| 7301/7301 [00:07<00:00, 976.57it/s]


In [39]:
accuracy = model.evaluate(X_test, y_test, verbose=1)
print('Test_Accuracy:-', accuracy[1])

3407/3407 [==============================] - 0s 129us/step
Test_Accuracy:- 0.7992368653123596


In [0]:
prediction = model.predict_classes(test)

In [0]:
download = drive.CreateFile({'id': '1qw8Lw8VRvExI-58C0wHD9O51iT6jN25I'})
download.GetContentFile('sample_submission_CH2mq5Z.csv')

In [0]:
sample = pd.read_csv('sample_submission_CH2mq5Z.csv')
sample['label']=prediction
sample.to_csv('sample_cnn.csv',header=True,index=False)

adc.json        sample_submission_CH2mq5Z.csv  train/
sample_cnn.csv  submission.csv                 train.csv
sample_data/    test_WyRytb0.csv               train-scene-classification.zip
